In [1]:
import pandas as pd
import os
import glob

project_root = os.path.abspath(os.getcwd())

In [18]:
#read in all years and combine
files = glob.glob(os.path.join(project_root, "PressFreedomIndex") + "/*.csv")

raw = []

for filename in files:
    df = pd.read_csv(filename, sep=';')
    raw.append(df)

raw = pd.concat(raw, axis=0, ignore_index=True)

raw.head()

,Year (N),ISO,Rank N,Score N,Score N without the exactions,Score N with the exactions,Score exactions,Rank N-1,Score N-1,Rank evolution,...,Rank_Leg,Social Context,Rank_Soc,Safety,Rank_Saf,Country_EN,Country_FR,Country_ES,Country_AR,Country_FA
0,2003,FIN,1.0,"0,5",NaN,NaN,NaN,1.0,"0,5",0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003,ISL,1.0,"0,5",NaN,NaN,NaN,2.0,"0,5",1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003,NOR,1.0,"0,5",NaN,NaN,NaN,3.0,"0,5",2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003,NLD,1.0,"0,5",NaN,NaN,NaN,4.0,"0,5",3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003,DNK,5.0,1,NaN,NaN,NaN,10.0,3,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
